# Load data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Set up the NNP and train

In [2]:
from mlearn.potentials.nnp import NNPotential

nnp = NNPotential()
nnp.train(train_structures=train_structures,
          energies=train_energies,
          forces=train_forces,
          cutoff_type=1, atom_energy=-4.14,
          r_etas = [0.5, 1.0, 2.0],
          r_shift=[0.0],
          zetas=[1.0, 2.0, 4.0],
          epochs = 10, r_cut=4.8, hidden_layers=[8, 8])

0

# Lattice constant, Elastic constant

In [3]:
from mlearn.potentials.lammps.calcs import LatticeConstant

conventional_cell = Structure.from_file('conventional.cif')
lc_calculator = LatticeConstant(ff_settings=nnp)
a, b, c = lc_calculator.calculate([conventional_cell])[0]
print('Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Lattice a: 3.11682098232346, Lattice b: 3.11682098232346, Lattice c: 3.11682098232346


In [4]:
from mlearn.potentials.lammps.calcs import ElasticConstant

ec_calculator = ElasticConstant(ff_settings=nnp, lattice='bcc', alat=3.106)
C11, C12, C44, bulk_modulus = ec_calculator.calculate()
print('C11: {}, C12: {}, C44: {}, bulk modulus: {}'.format(C11, C12, C44, bulk_modulus))

C11: 1891.08588157437, C12: 1583.51362646124, C44: 127.632843899908, bulk modulus: 1686.03771150194


# Load model from parameters files

In [5]:
from mlearn.potentials.nnp import NNPotential

nnp_loaded = NNPotential.from_config('input.nn', 'scaling.data', 'weights.data')

# Energy, force, stress prediction

In [6]:
from mlearn.potentials.lammps.calcs import EnergyForceStress
from pymatgen import Structure
struct = Structure.from_file('test_struct.cif')
efs_calculator = EnergyForceStress(ff_settings=nnp_loaded)
energy, forces, stresses = efs_calculator.calculate([struct])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -3734.08244351928
forces: 
 [[ 6.58465e+02  4.51711e+02 -6.75394e+02]
 [ 4.52466e+03 -1.17757e+03  7.77885e+02]
 [-4.92193e+01 -2.20098e+02  3.22273e+02]
 [-4.26189e+01 -7.03868e+01  3.04398e+01]
 [-3.35458e+02 -2.64674e+02 -5.31042e+00]
 [-4.25169e+02  7.45834e+02 -6.64879e+02]
 [-1.05633e+02  5.29713e+01  1.09211e+02]
 [ 3.93792e+02 -7.49834e+02  7.28043e+02]
 [ 4.06683e+02 -3.40036e+01  3.64031e+01]
 [-3.51473e+02 -5.04083e+03 -6.14042e+03]
 [ 5.24818e+01 -3.08489e+02 -6.68960e+01]
 [ 6.00224e+01  1.89148e+02  2.57944e+02]
 [-8.37456e+01  1.76310e+01 -2.21672e+01]
 [ 1.33050e+02 -4.06146e+01 -9.80264e+01]
 [ 3.23040e+02 -2.30522e+01 -2.18293e+02]
 [-5.90054e+02 -4.31821e+02  4.31893e+01]
 [ 4.40641e+00  8.75469e+01  2.70790e+01]
 [ 6.07223e+02  1.59326e+03  2.76147e+03]
 [ 1.15604e-02  3.29412e+01  1.51439e+02]
 [ 2.19256e+00  7.19628e+00 -1.00228e+02]
 [-2.60609e+02  4.14211e+02  6.63603e+01]
 [ 6.82806e+01  5.67329e+01  4.86627e+01]
 [-1.31569e+02  3.09083e+02  1.00032e+02